In [25]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from rasterio import features, open
from rasterio.transform import from_origin
from rasterio.plot import show
import rasterstats as rs
import rasterio
from shapely.geometry import box


In [23]:
def read_ref_raster(ref_raster_name):
    with rasterio.open(ref_raster_name) as ref:
        ref_transform = ref.transform
        ref_crs = ref.crs
        ref_width = ref.width
        ref_height = ref.height
    
    return (ref_transform, ref_crs, ref_width, ref_height)

def write_raster(output_name, raster, ref_height, ref_width, ref_crs, ref_transform):
    with rasterio.open(
        output_name,
        "w",
        driver="GTiff",
        height=ref_height,
        width=ref_width,
        count=1,
        dtype=raster.dtype,
        crs=ref_crs,
        transform=ref_transform,
        nodata=0
    ) as dst:
        dst.write(raster, 1)

In [24]:
def rasterize_binary(ref_raster_name, shapefile_name, output_name):        
    (ref_transform, ref_crs, ref_width, ref_height) = read_ref_raster(ref_raster_name)
    
    shapefile = gpd.read_file(shapefile_name)
    shapefile = shapefile.to_crs(ref_crs)
    
    shapes = ((geom, 1) for geom in shapefile.geometry)
    raster = features.rasterize(
        shapes,
        out_shape=(ref_height, ref_width),
        transform=ref_transform,
        fill=0,
        dtype=np.uint8
    )
    
    write_raster(output_name, raster, ref_height, ref_width, ref_crs, ref_transform)

In [ ]:
def pixel_area(ref_transform):
    return abs(ref_transform.a * ref_transform.e)

def pixel_bbox(row, col, ref_transform):
    pixel_width = abs(ref_transform.a)
    pixel_height = abs(ref_transform.e)
    
    x_left = ref_transform.c + col * pixel_width
    y_top = ref_transform.c + row * pixel_height
    x_right = x_left + pixel_width
    y_bottom = y_top + pixel_height
    return box(x_left, y_bottom, x_right, y_top)

def rasterize_interpolated(ref_raster_name, shapefile_name, attribute_name, output):
    (ref_transform, ref_crs, ref_width, ref_height) = read_ref_raster(ref_raster_name)

    shapefile = gpd.read_file(shapefile_name)
    shapefile = shapefile.to_crs(ref_crs)
    
    raster = np.zeros((ref_height, ref_width), dtype=np.float32)
    
    area = pixel_area(ref_transform)
    
    print('for begins')
    for row in range(ref_height):
        for col in range(ref_width):
            pixel = pixel_bbox(row, col, ref_transform)
            intersects = shapefile[shapefile.intersects(pixel)]
            for _, feat in intersects.iterrows():
                print('hello')
                inter_area = feat.geometry.intersection(pixel).area
                raster[row, col] += feat[attribute_name] * (inter_area / area)
    
    write_raster(output, raster, ref_height, ref_width, ref_crs, ref_transform)


In [ ]:
rasterize_interpolated("hague_light.tif", "hague_building_heights.shp", "rf_volume_", "hague_building_volumes.tif")